In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
%cd /content/LLM-course-2024/week-2/prompting-notebook
!ls

/content/LLM-course-2024/week-2/prompting-notebook
prompting.ipynb  requirements.txt


# Prompting a Large Language Model


This is an example notebook for running an open source model from Hugging Face.

In [8]:
%pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.4 MB/s eta 0:00:00


In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

We use the [Phi-3.5-mini-instruct](https://huggingface.co/microsoft/Phi-3.5-mini-instruct) in this notebook due to its small size.

In [10]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    device_map="cuda", # change to cpu is running locally
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

An example few-shot prompt:

In [11]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

In [12]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

Device set to use cuda


In [13]:
output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 To solve the equation 2x + 3 = 7, follow these steps:

Step 1: Isolate the term with the variable (2x) by subtracting 3 from both sides of the equation.
2x + 3 - 3 = 7 - 3
2x = 4

Step 2: Solve for x by dividing both sides of the equation by the coefficient of x, which is 2.
2x / 2 = 4 / 2
x = 2

So, the solution to the equation 2x + 3 = 7 is x = 2.


In [21]:
generation_args = {
    "max_new_tokens": 1000,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

system_prompt = "You are a professional psychologist with expertise in emotional analysis and counseling. Please generate information in a polite and formal manner."

# 1. Zero-shot prompting
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "I feel extremely happy today. Sometimes I worry this happiness won't last."},
]

output = pipe(messages, **generation_args)
print("Zero-shot result:\n", output[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Zero-shot result:
  It's wonderful to hear that you're experiencing such a positive state of mind. It's natural to have concerns about the transient nature of emotions, but it's important to remember that happiness, like all feelings, is a dynamic experience that can fluctuate over time. Here are a few thoughts and strategies that might help you appreciate and sustain your happiness:

1. **Embrace the Moment**: Try to live in the present and enjoy the happiness you're feeling now. Mindfulness practices can help you stay grounded and fully experience the joy in the here and now.

2. **Reflect on the Causes**: Consider what has contributed to your happiness today. Is it a particular event, a person, or perhaps a combination of factors? Understanding the source can help you replicate or build upon it.

3. **Set Realistic Expectations**: It's human to hope for continued happiness, but it's also important to set realistic expectations. Emotions are not constant, and it's okay to have ups an

In [18]:
# 2. Few-shot prompting
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "I feel sad"},
    {"role": "assistant", "content": "Analysis: Sadness often stems from loss or unmet expectations. Advice: Practice self-compassion and identify specific triggers."},
    {"role": "user", "content": "I feel anxious"},
    {"role": "assistant", "content": "Analysis: Anxiety typically relates to future uncertainties. Advice: Use grounding techniques and challenge negative thoughts."},
    {"role": "user", "content": "I feel extremely happy today. Sometimes I worry this happiness won't last."},
]

output = pipe(messages, **generation_args)
print("\nFew-shot result:\n", output[0]['generated_text'])


Few-shot result:
  Analysis: It's natural to have concerns about the sustainability of happiness. Happiness is a complex emotion influenced by various factors, including personal achievements, relationships, and overall well-being.

Here are some suggestions to help you maintain your happiness:

1. Acknowledge and appreciate: Recognize the reasons behind your happiness and take time to appreciate them. This can help you understand what brings you joy and make it a part of your life.

2. Practice mindfulness: Mindfulness can help you stay present and enjoy the moment. It can also help you manage your worries about the future.

3. Set realistic expectations: Understand that happiness is not a constant state. It's okay to have ups and downs. Accepting this can help you maintain your happiness even when it fluctuates.

4. Cultivate gratitude: Regularly reflecting on the things you're grateful for can help shift your focus from what you lack to what you have, promoting a more positive outl

In [19]:
# 3. Chain-of-thought prompting
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": """Let's analyze this emotion step by step:
"I feel extremely happy today. Sometimes I worry this happiness won't last."

1. First, let's identify the primary emotion
2. Then, examine any secondary emotions or concerns
3. Next, analyze potential triggers or causes
4. Consider the cognitive patterns involved
5. Finally, develop appropriate recommendations

Please think through each step and provide your analysis."""},
]

output = pipe(messages, **generation_args)
print("\nChain-of-thought result:\n", output[0]['generated_text'])


Chain-of-thought result:
  1. Primary Emotion Identification:
The primary emotion expressed in the statement is happiness. The individual clearly states that they are "extremely happy today," which indicates a strong positive emotional state.

2. Secondary Emotions or Concerns:
The secondary emotion or concern expressed in the statement is worry. The individual worries that their happiness may not last, which introduces a sense of uncertainty and potential anxiety about the future.

3. Potential Triggers or Causes:
Several potential triggers or causes could be contributing to the individual's happiness and subsequent worry. These may include:

   a. Achieving a personal goal or milestone
   b. Receiving positive feedback or recognition
   c. Spending time with loved ones or engaging in enjoyable activities
   d. A sense of accomplishment or progress in a personal or professional endeavor

   The worry about the happiness not lasting could be triggered by:

   a. Fear of losing the pos

In [23]:
# 4. Prompt chaining
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Analyze this emotional state and identify key components: I feel extremely happy today. Sometimes I worry this happiness won't last."},
]

output1 = pipe(messages, **generation_args)
analysis = output1[0]['generated_text']

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "assistant", "content": analysis},
    {"role": "user", "content": "Based on this analysis, provide specific, actionable recommendations for maintaining emotional well-being."},
]

output = pipe(messages, **generation_args)
print("\nPrompt chaining result:\n", output[0]['generated_text'])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Prompt chaining result:
  Certainly, here are some specific, actionable recommendations to help maintain your emotional well-being:

1. **Maintain a Gratitude Journal**: Start a daily practice of writing down things you're grateful for. This can help shift your focus from uncertainty to appreciation, fostering a positive emotional state.

2. **Engage in Regular Physical Activity**: Exercise releases endorphins, which are known as "feel-good" hormones. Incorporate at least 30 minutes of physical activity into your daily routine. This could be anything from a brisk walk, yoga, or a dance class.

3. **Practice Mindfulness and Meditation**: These practices can help you stay present and accept the transient nature of emotions. There are numerous apps and online resources available to guide you through mindfulness exercises and meditation.

4. **Set Realistic Goals**: If your happiness is tied to achieving something, set realistic goals and break them down into manageable steps. Celebrate s

In [24]:
# 5. Tree-of-thought prompting
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": """Let's analyze this emotional state through multiple perspectives:
"I feel extremely happy today. Sometimes I worry this happiness won't last."

1. Consider three possible sources of the happiness:
A) External circumstances
B) Internal psychological state
C) Physiological factors

2. For each source, explore:
- Potential triggers
- Sustainability factors
- Associated risks

3. Evaluate which explanation(s) seem most likely

4. Develop personalized recommendations based on the most probable scenario"""},
]

output = pipe(messages, **generation_args)
print("\nTree-of-thought result:\n", output[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Tree-of-thought result:
  1. Analysis of the Emotional State:

The individual expresses a sense of extreme happiness, yet there is an underlying concern that this happiness may not be enduring. To understand this emotional state more comprehensively, we will examine it from three different perspectives: external circumstances, internal psychological state, and physiological factors.

2. Exploration of Sources:

A) External Circumstances:

- Potential Triggers: Positive life events such as achieving a personal goal, receiving praise or recognition, or spending time with loved ones.
- Sustainability Factors: The continuity of these positive experiences and the ability to create a supportive environment that fosters happiness.
- Associated Risks: Over-reliance on external factors for happiness can lead to disappointment if these factors change or cease to exist.

B) Internal Psychological State:

- Potential Triggers: A sense of accomplishment, self-acceptance, or a positive outlook on l